Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

Function to create the custom DataFrame

In [ ]:
def create_custom_df(start_date, end_date, tickers, company_names, days):
    # Convert start_date and end_date to datetime64[ns]
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filter days DataFrame to the specified date range
    filtered_days = days[(days['Date'] >= start_date) & (days['Date'] <= end_date)].copy()

    # Initialize the custom DataFrame with the filtered days
    custom_df = pd.DataFrame(filtered_days['Date'], columns=['Date'])

    # Ensure the 'Date' column in the custom_df and all relevant DataFrames is datetime64[ns]
    custom_df['Date'] = pd.to_datetime(custom_df['Date'])
    
    # Add market cap data for the specific tickers
    for ticker, name in zip(tickers, company_names):
        ticker_data = yf.download(ticker, start=start_date, end=end_date)
        if 'Date' not in ticker_data.columns:
            ticker_data.reset_index(inplace=True)
        print(f"Merging Market Cap {ticker} ({name}) with {len(ticker_data)} rows")
        custom_df = custom_df.merge(ticker_data[['Date', 'Close']], on='Date', how='left')
        custom_df.rename(columns={'Close': name}, inplace=True)

    # Remove rows where all elements are NaN except the 'Date' column
    custom_df = custom_df.dropna(how='all', subset=custom_df.columns.difference(['Date']))

    return custom_df

Function to normalize the DataFrame

In [ ]:
def normalize_dataframe(df):
    normalized_df = df.copy()
    for column in df.columns:
        if column != 'Date':
            initial_index = df[column].first_valid_index()
            if initial_index is None:
                print(f"No valid data for {column}. Skipping normalization for this column.")
                normalized_df[column] = np.nan
            else:
                initial_value = df.at[initial_index, column]
                normalized_df[column] = (df[column] / initial_value) * 100
                normalized_df.at[initial_index, column] = 100  # Ensure the first valid value is set to 100
    print(f"Normalized DataFrame:\n{normalized_df.head()}")
    return normalized_df

Create a DataFrame with All Dates

In [ ]:
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2024, 6, 1)
days = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='D', name='Date'))
days['Date'] = pd.to_datetime(days['Date'])

List of Tickers and Company Names

In [ ]:
tickers = ['SPNS', 'REAX', 'PERI', 'NVMI', 'INMD', 'CYBR', 'CAMT', 'WIX']
company_names = [
    'Sapiens International Corporation',
    'Real',
    'Perion Network',
    'Nova Measuring Instruments',
    'InMode',
    'CyberArk',
    'Camtek',
    'Wix'
]

Create the custom DataFrame

In [35]:
custom_df = create_custom_df(start_date, end_date, tickers, company_names, days)

print(f"Custom DataFrame before normalization:\n{custom_df.head()}")
normalized_df = normalize_dataframe(custom_df)

# Save the custom DataFrame to an Excel file
output_file_path = 'custom_data_with_company_names.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    custom_df.to_excel(writer, sheet_name='Custom Data', index=False)
    normalized_df.to_excel(writer, sheet_name='Normalized Data', index=False)

print(f"DataFrames have been saved to {output_file_path}")


[*********************100%%**********************]  1 of 1 completed


Merging Market Cap SPNS (Sapiens International Corporation) with 105 rows


[*********************100%%**********************]  1 of 1 completed


Merging Market Cap REAX (Real) with 105 rows


[*********************100%%**********************]  1 of 1 completed


...

Merging Market Cap CYBR (CyberArk) with 105 rows


[*********************100%%**********************]  1 of 1 completed


Merging Market Cap CAMT (Camtek) with 105 rows


[*********************100%%**********************]  1 of 1 completed

Merging Market Cap WIX (Wix) with 105 rows
Custom DataFrame before normalization:
        Date  Sapiens International Corporation   Real  Perion Network  \
1 2024-01-02                          28.049999  1.540       30.330000   
2 2024-01-03                          26.969999  1.650       29.510000   
3 2024-01-04                          26.570000  1.790       29.139999   
4 2024-01-05                          26.719999  1.770       28.760000   
7 2024-01-08                          27.510000  1.765       29.010000   

   Nova Measuring Instruments     InMode    CyberArk     Camtek         Wix  
1                  135.649994  22.219999  216.130005  67.459999  118.000000  
2                  132.100006  21.000000  214.380005  67.019997  117.400002  
3                  131.960007  21.010000  211.679993  68.620003  116.900002  
4                  129.320007  20.959999  211.720001  67.930000  118.099998  
7                  131.460007  21.370001  217.990005  68.750000  120.459999  
Norma

Saving DataFrames to Excel

In [ ]:
# Save the custom DataFrame and normalized DataFrame to a new Excel file with a graph
output_path = 'C:\\Users\\odeya.h\\SNC Dropbox\\Odeya Hazani Cohen\\פרוייקט\\top_compnies_data_2024.xlsx'
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    custom_df.to_excel(writer, sheet_name='Custom Data', index=False)
    normalized_df.to_excel(writer, sheet_name='Normalized Data', index=False)  

In [36]:
    # Create a chart object
    workbook  = writer.book
    worksheet = writer.sheets['Normalized Data']
    chart = workbook.add_chart({'type': 'line'})

    # Configure the series of the chart from the DataFrame data
    for i, column in enumerate(normalized_df.columns[1:]):  # Skip the 'Date' column
        chart.add_series({
            'name':       [worksheet.name, 0, i + 1],
            'categories': [worksheet.name, 1, 0, len(normalized_df), 0],
            'values':     [worksheet.name, 1, i + 1, len(normalized_df), i + 1],
        })

    # Configure the chart title with professional formatting
    chart.set_title({
        'name': 'Index Comparison',
        'name_font': {
            'bold': True,
            'size': 14,
            'name': 'Arial'
        }
    })

    # Configure the chart axes with professional formatting and the new date format
    chart.set_x_axis({
        'name': 'Date',
        'date_axis': True,
        'num_format': 'dd-mm-yyyy',
        'name_font': {
            'bold': True,
            'size': 12,
            'name': 'Arial'
        },
        'label_font': {
            'size': 10,
            'name': 'Arial'
        },
        'major_gridlines': {
            'visible': True,
            'line': {'width': 0.75, 'dash_type': 'dash'}
        },
        'minor_gridlines': {
            'visible': True,
            'line': {'width': 0.25, 'dash_type': 'dash'}
        }
    })
    chart.set_y_axis({
        'name': 'Normalized Value',
        'name_font': {
            'bold': True,
            'size': 12,
            'name': 'Arial'
        },
        'label_font': {
            'size': 10,
            'name': 'Arial'
        },
        'major_gridlines': {
            'visible': True,
            'line': {'width': 0.75, 'dash_type': 'dash'}
        },
        'minor_gridlines': {
            'visible': True,
            'line': {'width': 0.25, 'dash_type': 'dash'}
        }
    })

    # Customize the legend
    chart.set_legend({
        'position': 'bottom',
        'font': {
            'size': 10,
            'name': 'Arial'
        }
    })

    # Adjust the chart size and positioning
    worksheet.insert_chart('G2', chart, {'x_offset': 25, 'y_offset': 10, 'x_scale': 1.5, 'y_scale': 1.5})

print(f"Custom data with graph has been saved to {output_path}")


Custom data with graph has been saved to C:\Users\odeya.h\SNC Dropbox\Odeya Hazani Cohen\פרוייקט\top_compnies_data_2024.xlsx
